### SVM

Implemantação do classificador utilizando SVM.

In [1]:
import nltk
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from string import ascii_lowercase

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report

from pymongo import MongoClient

%matplotlib inline

In [2]:
'''
Importação dos dados da base
'''
client = MongoClient()
client = MongoClient('localhost', 27800)
db = client.network_logs
db_unique_hostnames = db.unique_hostnames

df_hostnames = pd.DataFrame(list(db_unique_hostnames.find({})))

print(len(df_hostnames))
df_hostnames.head()

24208


,_id,catdesc,category,hostname,normalized_hostname,normalized_hostname_url,normalized_url,service,url
0,59a1e31988b032071ef8c495,Information Technology,Information Technology,sb.scorecardresearch.com,scorecardresearch,scorecardresearch,,HTTPS,/
1,59a1e31a88b032071ef8c496,Information Technology,Information Technology,test.com,test,testpartneruidUPcbedebee,partneruidUPcbedebee,HTTP,/?partner_uid=UP47c5b905-0e1d-11e7-b092-00163e...
2,59a1e31a88b032071ef8c497,Information Technology,Information Technology,nexus-websocket-b.intercom.io,nexuswebsocketbintercom,nexuswebsocketbintercom,,HTTPS,/
3,59a1e31a88b032071ef8c498,Malicious Websites,Malicious or Illegal,aorta.clickagy.com,aortaclickagy,aortaclickagypixelgifadvertiseridgtqxdqpglista...,pixelgifadvertiseridgtqxdqpglistaedjpnobncchcm...,HTTP,/pixel.gif?advertiser_id=1gt8qx97dqpg&list=11a...
4,59a1e31a88b032071ef8c499,Information Technology,Information Technology,usersync.videoamp.com,usersyncvideoamp,usersyncvideoampusersyncpartneridpartneruserid...,usersyncpartneridpartneruseridredirecthttpsync...,HTTP,/usersync?partner_id=5797361&partner_user_id=4...


In [3]:
'''
Criação do vocabulário
'''
keywords = [''.join(trig) for trig in itertools.product(ascii_lowercase, repeat = 3)]
len(keywords)

17576

### Vetorização e frequência dos termos

In [4]:
count_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3), vocabulary=keywords)
words_vector = count_vectorizer.transform(df_hostnames['normalized_hostname'])
tf_transformer = TfidfTransformer(use_idf=False).fit(words_vector)
urls_tf = tf_transformer.transform(words_vector)

### Separação do conjunto

In [5]:
labels = df_hostnames['category']
url_train,url_test,label_train,label_test = train_test_split(urls_tf, labels, test_size=0.3)

### Classificação e teste

In [6]:
classifier_model = LinearSVC().fit(url_train, label_train)

In [7]:
predictions = classifier_model.predict(url_test)

### Relatório de classificação

In [8]:
print(classification_report(label_test, predictions))

                                 precision    recall  f1-score   support

                          Adult       0.00      0.00      0.00        20
                    Advertising       0.56      0.52      0.54       295
               Arts and Culture       0.57      0.30      0.40        56
                       Business       0.35      0.39      0.37       687
                Content Servers       0.80      0.74      0.77       361
                 Domain Parking       0.67      0.11      0.19        18
                      Education       0.71      0.52      0.60       256
                  Entertainment       0.33      0.22      0.26       140
       File Sharing and Storage       0.86      0.92      0.89       143
            Finance and Banking       0.57      0.32      0.41        62
Freeware and Software Downloads       0.53      0.41      0.46        56
                          Games       0.61      0.62      0.61       190
            Health and Wellness       0.47      0.